In [2]:
import pandas as pd
import numpy as np

# 시간 범위 설정
start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2023-12-31")
num_days = (end_date - start_date).days + 1

# 태양광 데이터 생성 함수
def generate_solar_data(start_date, end_date, num_days):
    # 태양광 데이터 생성을 위한 평균과 표준편차 설정
    mean_irradiance = 500  # 태양광 조도 평균
    std_irradiance = 50    # 태양광 조도 표준편차
    mean_temperature = 25  # 온도 평균
    std_temperature = 5    # 온도 표준편차
    mean_humidity = 50     # 습도 평균
    std_humidity = 10      # 습도 표준편차
    mean_rainfall = 5      # 강우량 평균
    std_rainfall = 2       # 강우량 표준편차
    mean_DHI = 300         # DHI 평균
    std_DHI = 100          # DHI 표준편차
    mean_DNI = 500         # DNI 평균
    std_DNI = 150          # DNI 표준편차

    # 랜덤 시드 설정
    np.random.seed(0)

    # 날짜별 데이터 생성
    data_list = []
    for i in range(num_days):
        # 날짜 계산
        date = start_date + pd.Timedelta(days=i)
        
        # 평균과 표준편차를 이용하여 실제 데이터와 유사한 분포를 가진 데이터 생성
        irradiance = np.random.normal(mean_irradiance, std_irradiance)
        temperature = np.random.normal(mean_temperature, std_temperature)
        humidity = np.random.normal(mean_humidity, std_humidity)
        rainfall = np.random.normal(mean_rainfall, std_rainfall)
        DHI = np.random.normal(mean_DHI, std_DHI)
        DNI = np.random.normal(mean_DNI, std_DNI)
        
        # Defect 여부 판단 (임의로 생성)
        defect = np.random.choice([0, 1])  # 불량 여부를 0 또는 1로 랜덤하게 생성
        
        # 데이터 리스트에 추가
        data_list.append({"date": date, "temperature": temperature, "humidity": humidity,
                          "irradiance": irradiance, "rainfall": rainfall, "DHI": DHI, "DNI": DNI,
                          "defect": defect})

    return data_list

# 데이터 생성
data_list = generate_solar_data(start_date, end_date, num_days)

# 데이터프레임 생성
data = pd.DataFrame(data_list)

# 데이터 저장
data.to_csv("solar_data.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

# 데이터 불러오기
data = pd.read_csv("data.csv")

# Feature와 Target 분리
X = data.drop(['date', 'power'], axis=1)  # 'date'와 'power'는 feature로 사용하지 않음
y = (data['power'] > data['power'].mean()).astype(int)  # 평균 이상인 경우 1, 이하인 경우 0으로 분류

# 데이터 스케일링 (두 가지 방법: 표준화, 정규화)
def scale_data(X, scaler_type='standard'):
    if scaler_type == 'standard':
        scaler = StandardScaler()
    elif scaler_type == 'minmax':
        scaler = MinMaxScaler()
    else:
        raise ValueError("Invalid scaler type. Choose between 'standard' and 'minmax'.")
    
    X_scaled = scaler.fit_transform(X)
    return X_scaled

# 모델 학습 및 평가
def train_and_evaluate(X, y, scaler_type='standard', model='random_forest', param_grid=None):
    # 데이터 스케일링
    X_scaled = scale_data(X, scaler_type)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # 모델 선택
    if model == 'random_forest':
        clf = RandomForestClassifier(random_state=42)
    elif model == 'decision_tree':
        clf = DecisionTreeClassifier(random_state=42)
    elif model == 'xgboost':
        clf = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    else:
        raise ValueError("Invalid model type. Choose between 'random_forest', 'decision_tree', and 'xgboost'.")
    
    # 그리드 서치 수행
    if param_grid:
        grid_search = GridSearchCV(clf, param_grid, cv=5)
        grid_search.fit(X_train, y_train)
        best_clf = grid_search.best_estimator_
        print("Best parameters:", grid_search.best_params_)
    else:
        best_clf = clf.fit(X_train, y_train)
    
    # 평가
    train_accuracy = best_clf.score(X_train, y_train)
    test_accuracy = best_clf.score(X_test, y_test)
    
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    
    return best_clf

# 그리드 서치 함수
def perform_grid_search(X, y, model='random_forest', param_grid=None):
    if model == 'random_forest':
        clf = RandomForestClassifier(random_state=42)
    elif model == 'decision_tree':
        clf = DecisionTreeClassifier(random_state=42)
    elif model == 'xgboost':
        clf = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
    else:
        raise ValueError("Invalid model type. Choose between 'random_forest', 'decision_tree', and 'xgboost'.")
    
    grid_search = GridSearchCV(clf, param_grid, cv=5)
    grid_search.fit(X, y)
    
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)
    return grid_search.best_estimator_

# 예시: 랜덤 포레스트 모델 학습 및 평가
best_rf_model = train_and_evaluate(X, y, scaler_type='standard', model='random_forest', param_grid={'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]})
